In [8]:
from cfg.configfile_req import *
from lib.csysrq_support import *
from redminelib import Redmine

redmine = Redmine(req_server_url,key=req_key_txt)
projects = redmine.project.all()

print("Proyectos:")
for p in projects:
    print ("\t",p.identifier," \t| ",p.name)
    

Proyectos:
	 reqtest  	|  Requirements test


In [9]:
my_project = redmine.project.get(req_project_id_str)
print ("Obtenemos proyecto: ",my_project.identifier," | ",my_project.name)

Obtenemos proyecto:  reqtest  |  Requirements test


In [10]:
tmp = redmine.issue.filter(project_id=req_project_id_str, tracker_id=req_rq_tracker_id)
my_project_issues = sorted(tmp, key=lambda k: k.subject)
tmp = redmine.issue.filter(project_id=req_project_id_str, tracker_id=req_doc_tracker_id)
my_doc_issues = sorted(tmp, key=lambda k: k.subject)

Ahora conectamos con DOORSTOP

In [11]:
import doorstop

doorstop.__version__

'1.5.1'

Creamos la raíz de los requisitos en doorstop

Ahora generaremos los documentos a partir de aquellos requisitos que tengan algún hijo

In [12]:
tree = doorstop.build()

rootIssue = True
for my_issue in my_doc_issues:
    print("********** ",my_issue)
    if rootIssue:
        docpath = "./doorstop/"+my_issue.subject
        prefix = my_issue.subject+"-"
        doc = tree.create_document(docpath,prefix)
        rootIssue = False
    else:
                prefix = my_issue.subject+"-"           
                parent_issue = redmine.issue.get(my_issue.parent.id)
                #print("parent: ",parent_issue.subject)
                docpath = "./doorstop/"+my_issue.subject
                iss = redmine.issue.get(my_issue.id)
                print("prefix: ",prefix)
                doc = tree.create_document(docpath,prefix,parent=parent_issue.subject+"-")


**********  SR
**********  SRC
prefix:  SRC-


/home/cosmobots/anaconda3/lib/python3.7/site-packages/NRT_Doorstop-1.5.1-py3.7.egg/doorstop/common.py:126: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(text) or {}
/home/cosmobots/anaconda3/lib/python3.7/site-packages/NRT_Doorstop-1.5.1-py3.7.egg/doorstop/common.py:126: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(text) or {}


Ahora crearemos los requisitos "hijos" dentro de cada documento

In [13]:
def find_prefix(this_issue):
    if this_issue.tracker.id== req_doc_tracker_id:
        return(this_issue.subject+"-")

    # not solved yet
    parent_issue = redmine.issue.get(this_issue.parent.id)
    print("Llamo al padre")
    return find_prefix(parent_issue)

from doorstop.core.item import Item

for my_issue in my_project_issues:
    reqname = my_issue.subject
    print("reqname: ",reqname)
    parent_issue = redmine.issue.get(my_issue.parent.id)
    print("parent: ",parent_issue.subject)
    prefix = find_prefix(parent_issue)
    if (prefix is not None):
        print("prefix: ",prefix)        
        #newitem = tree.add_item(prefix)
        #newitem.text = my_issue.description

        document = tree.find_document(prefix)
        newitem = Item.new(tree, document,
                document.path, document.root, my_issue.subject,
                auto=False)
        newitem.set('RqTitle', my_issue.custom_fields.get(req_title_cf_id).value)
        newitem.set('RqLevel', my_issue.custom_fields.get(req_level_cf_id).value)
        newitem.set('RqType', my_issue.custom_fields.get(req_type_cf_id).value)
        newitem.set('RqSrc', my_issue.custom_fields.get(req_sources_cf_id).value)
        newitem.set('ReqRationale', my_issue.custom_fields.get(req_rationale_cf_id).value)
        newitem.set('text', my_issue.description)
        newitem.save()
        print("---->",newitem)


reqname:  SR-0001
parent:  SR
prefix:  SR-
----> SR-0001
reqname:  SR-0002
parent:  SR-0001
Llamo al padre
prefix:  SR-
----> SR-0002
reqname:  SR-0003
parent:  SR
prefix:  SR-
----> SR-0003
reqname:  SR-0004


/home/cosmobots/anaconda3/lib/python3.7/site-packages/NRT_Doorstop-1.5.1-py3.7.egg/doorstop/common.py:126: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(text) or {}
/home/cosmobots/anaconda3/lib/python3.7/site-packages/NRT_Doorstop-1.5.1-py3.7.egg/doorstop/common.py:126: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(text) or {}


parent:  SR-0003
Llamo al padre
prefix:  SR-
----> SR-0004
reqname:  SR-0005
parent:  SR-0004


/home/cosmobots/anaconda3/lib/python3.7/site-packages/NRT_Doorstop-1.5.1-py3.7.egg/doorstop/common.py:126: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(text) or {}


Llamo al padre
Llamo al padre
prefix:  SR-
----> SR-0005
reqname:  SR-0006


...

/home/cosmobots/anaconda3/lib/python3.7/site-packages/NRT_Doorstop-1.5.1-py3.7.egg/doorstop/common.py:126: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(text) or {}


parent:  SRC-0004
Llamo al padre
Llamo al padre
prefix:  SRC-
----> SRC-0008
reqname:  SRC-0009


/home/cosmobots/anaconda3/lib/python3.7/site-packages/NRT_Doorstop-1.5.1-py3.7.egg/doorstop/common.py:126: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(text) or {}
/home/cosmobots/anaconda3/lib/python3.7/site-packages/NRT_Doorstop-1.5.1-py3.7.egg/doorstop/common.py:126: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(text) or {}


parent:  SRC-0003
Llamo al padre
prefix:  SRC-
----> SRC-0009
reqname:  SRC-0010
parent:  SRC-0009
Llamo al padre
Llamo al padre
prefix:  SRC-
----> SRC-0010


/home/cosmobots/anaconda3/lib/python3.7/site-packages/NRT_Doorstop-1.5.1-py3.7.egg/doorstop/common.py:126: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(text) or {}


In [14]:
# Creamos los links entre requisitos del Redmine
for i in my_project_issues:
    thisitem = tree.find_item(i.subject)
    if (thisitem is not None):
        print(" thisitem: ",thisitem)

    # Busco las relaciones en las que es destinatario
    my_issue_relations = redmine.issue_relation.filter(issue_id=i.id)
    my_filtered_issue_relations = list(filter(lambda x: x.issue_id != i.id, my_issue_relations))

    # Recorro las relaciones creando los links
    for rel in my_filtered_issue_relations:
        # Obtenemos la incidencia y el item doorstop del objeto que es origen de la relación de Redmine,
        # que significa que es destinatario de la relación de doorstop, ya que es el elemento que está
        # condicionando al actual (el actual depende de él)
        relissue = redmine.issue.get(rel.issue_id)
        print("Relacionado: ",rel," de ",relissue.subject," a ",i.subject)
        relitem = tree.find_item(relissue.subject)
        # Creamos un link donde expresamos que thisitem depende de relitem
        thisitem.link(relitem)

    
print("Acabamos")


 thisitem:  SR-0001
 thisitem:  SR-0002


/home/cosmobots/anaconda3/lib/python3.7/site-packages/NRT_Doorstop-1.5.1-py3.7.egg/doorstop/common.py:126: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(text) or {}


 thisitem:  SR-0003
 thisitem:  SR-0004
 thisitem:  SR-0005
 thisitem:  SR-0006
 thisitem:  SR-0007
Relacionado:  1  de  SR-0005  a  SR-0007
Relacionado:  2  de  SR-0006  a  SR-0007
 thisitem:  SR-0008
Relacionado:  3  de  SR-0005  a  SR-0008
 thisitem:  SR-0009
 thisitem:  SR-0010


/home/cosmobots/anaconda3/lib/python3.7/site-packages/NRT_Doorstop-1.5.1-py3.7.egg/doorstop/common.py:126: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(text) or {}
/home/cosmobots/anaconda3/lib/python3.7/site-packages/NRT_Doorstop-1.5.1-py3.7.egg/doorstop/common.py:126: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(text) or {}
/home/cosmobots/anaconda3/lib/python3.7/site-packages/NRT_Doorstop-1.5.1-py3.7.egg/doorstop/common.py:126: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(text) or {}


 thisitem:  SRC-0001
 thisitem:  SRC-0002
 thisitem:  SRC-0003
 thisitem:  SRC-0004
 thisitem:  SRC-0005
Relacionado:  4  de  SR-0004  a  SRC-0005
 thisitem:  SRC-0006
 thisitem:  SRC-0007
Relacionado: 

/home/cosmobots/anaconda3/lib/python3.7/site-packages/NRT_Doorstop-1.5.1-py3.7.egg/doorstop/common.py:126: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(text) or {}


 5  de  SRC-0005  a  SRC-0007
Relacionado:  6  de  SRC-0006  a  SRC-0007
 thisitem:  SRC-0008


/home/cosmobots/anaconda3/lib/python3.7/site-packages/NRT_Doorstop-1.5.1-py3.7.egg/doorstop/common.py:126: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(text) or {}
/home/cosmobots/anaconda3/lib/python3.7/site-packages/NRT_Doorstop-1.5.1-py3.7.egg/doorstop/common.py:126: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(text) or {}


Relacionado:  7  de  SRC-0005  a  SRC-0008
 thisitem:  SRC-0009
 thisitem:  SRC-0010
Acabamos


/home/cosmobots/anaconda3/lib/python3.7/site-packages/NRT_Doorstop-1.5.1-py3.7.egg/doorstop/common.py:126: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(text) or {}
